In [1]:
import neal
import numpy as np
from numpy import linalg as LA

ModuleNotFoundError: No module named 'neal'

Here we define the dimension of the matrix we have and the number of bits used in the computation

In [2]:
dim_matrix=2
num_of_bits=2

Define p vector and P matrix

In [3]:
p=np.empty([num_of_bits])
p[0]=-1
for b in range(2,num_of_bits+1):
    p[b-1]=2**(-b+1) 
P=np.empty([dim_matrix,dim_matrix*num_of_bits])
P[:][:]=0
for i in range(dim_matrix):
    P[i][i*num_of_bits:(i+1)*num_of_bits]=p[:]

Writing the array in the binary basis

In [7]:
def bin_matrix(A):
    QA=np.matmul(P.T,np.matmul(A,P))
    return QA

Random array A. The matrix we want diagonalize!

In [8]:
I=np.eye(dim_matrix,dim_matrix)
A=10*(np.random.rand(dim_matrix,dim_matrix)-0.5)
for i in range(dim_matrix):
    for j in range(dim_matrix):
        A[j][i]=A[i][j]
print(A)
w,v=LA.eigh(A)
print(I,w,v)

[[ 4.85105024 -1.2476392 ]
 [-1.2476392   1.48197036]]
[[1. 0.]
 [0. 1.]] [1.07025688 5.26276372] [[-0.31337233 -0.94963034]
 [-0.94963034  0.31337233]]


Compute trace

In [9]:
lam=np.trace(A)/dim_matrix
print(lam)
QA=bin_matrix(A-lam*I)
print(QA)

3.1665102998178027
[[ 1.68453994 -0.84226997 -1.2476392   0.6238196 ]
 [-0.84226997  0.42113499  0.6238196  -0.3119098 ]
 [-1.2476392   0.6238196  -1.68453994  0.84226997]
 [ 0.6238196  -0.3119098   0.84226997 -0.42113499]]
